In [13]:
import pandas as pd
import numpy as np
import pyproj
from pyproj import Transformer, CRS
from data_processing_for_features import *

### Interpolating trajectory points

<b>Aim</b>: To interpolate missing trajectory points </br>
<b>Challenge</b>: Python pandas interpolate functionality performs a linear interpolation whereas, the GPS system is a spherical coordinate system so using pandas for interpolation may not do correct interpolation </br>
<b>Solution</b>: </br> 
1. We first find in which coordinate reference system (CRS) are our coordinates recorded (most its in in 4326, if not, bring it in this format. If its already in 27700, then no need to do Step 2) 
2. Convert them in the a planar coordinate system like 27700 for instance
3. Perform a linear interpolation on the converted coordinates
4. Reconvert them to the spherical system and overwrite your lat/lon dataframe columns

In [69]:
cs_gt_df = pd.read_csv('../data_files/cabs_traj_GT.txt')
cs_test_df = pd.read_csv('../data_files/cabs_traj_test_orig.txt')
cs_test_df = cs_test_df[['latitude','longitude']]
cs_test_df.head(20)

,latitude,longitude
0,37.78506,-122.40026
1,37.78666,-122.39856
2,37.78676,-122.39851
3,NaN,-122.39916
4,37.78911,-122.40205
5,37.78911,-122.40205
6,37.79541,-122.40336
7,37.79640,-122.40468
8,37.79732,NaN
9,37.79912,-122.40875


In [62]:
# fetch indexes of rows where there is atleast a null value
lst_indexes_with_lat_nans = cs_test_df[cs_test_df['latitude'].isnull()].index.tolist()
lst_indexes_with_long_nans = cs_test_df[cs_test_df['longitude'].isnull()].index.tolist()
#print(lst_indexes_with_lat_nans)
#print(lst_indexes_with_long_nans)
lst_indexes_with_nans = lst_indexes_with_lat_nans + lst_indexes_with_long_nans
lst_indexes_with_nans = list(set(lst_indexes_with_nans))
# print(lst_indexes_with_nans)

In [64]:
# method to convert from spherical to planar coordinates
def fetch_planar_coord(lat, long, transformer):
    if np.isnan(lat) or np.isnan(long):
        x,y = np.nan, np.nan    
    else:
        x,y = transformer.transform(lat,long)
    concat_xy = str(x)+','+str(y)
    return concat_xy

In [65]:
def impute_df(df):
    df['x'] = 0
    df['y'] = 0
    df['concat_xy'] = ""
    transformer = Transformer.from_crs(4326, 27700) # transformer to perform Step 1 of solution
    back_transformer = Transformer.from_crs(27700, 4326) # back-transformer to perform Step 4 of solution
    
    
    df['concat_xy'] = df.apply(lambda row: fetch_planar_coord(row['latitude'], row['longitude'], transformer), axis=1) #indexes are not counted and numbering starts from 0, -1 means the last column
    df['x'] = df['concat_xy'].apply(lambda x: float(x.split(',')[0]) if x.split(',')[0] != 'nan' else np.nan)
    df['y'] = df['concat_xy'].apply(lambda y: float(y.split(',')[1]) if y.split(',')[1] != 'nan' else np.nan)   
    df = df[['latitude','longitude','x','y']]
    
    ## Step 3: interpolating missing values in planar coordinates
    df = df.interpolate(method='linear')
    
    
    ## Step 4: reconverting to spherical coordinates
    lat_back, lon_back = back_transformer.transform(df.x.values, df.y.values)
    df.latitude = lat_back
    df.longitude = lon_back
    
    df = df[['latitude','longitude']]
    
    return df

In [66]:
new_cs_test_df = impute_df(cs_test_df)
new_cs_test_df.head(20)

,latitude,longitude
0,37.78506,-122.40026
1,37.78666,-122.39856
2,37.78676,-122.39851
3,37.78761,-122.39916
4,37.78911,-122.40205
5,37.78911,-122.40205
6,37.79541,-122.40336
7,37.79640,-122.40468
8,37.79732,-122.40602
9,37.79912,-122.40875


### Evaluation

In [56]:
# find great circle distance between two points
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    All args must be of equal length.    
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    m = 6367000 * c
    return m

In [67]:
lst_distances = []
for idx in lst_indexes_with_nans:
    lst_distances.append(haversine_np(cs_gt_df.loc[idx,'longitude'],cs_gt_df.loc[idx,'latitude'],new_cs_test_df.loc[idx,'longitude'],new_cs_test_df.loc[idx,'latitude']))

In [68]:
print(sum(lst_distances)/len(lst_distances))

1.1112511235873423
